# Equivalent Operation of Vector/Matrix Calculation

- LibTorch
- oneMKL
- XTensor

In [1]:
#include <iostream>
#include <vector>

In [2]:
!python -c 'import torch;print(torch.utils.cmake_prefix_path)'

/opt/conda/lib/python3.10/site-packages/torch/share/cmake


In [3]:
// [cauachagas/cling-torch: Tutorial de como criar um ambiente de desenvolvimento para usar Jupyter com a API C++ do Pytorch.](https://github.com/cauachagas/cling-torch)
#pragma cling add_include_path("/opt/conda/lib/python3.10/site-packages/torch/include")
#pragma cling add_include_path("/opt/conda/lib/python3.10/site-packages/torch/include/torch/csrc/api/include")
#pragma cling add_library_path("/opt/conda/lib/python3.10/site-packages/torch/lib")
#pragma cling load("libtorch")

In [4]:
#include <torch/torch.h>

torch::Tensor libtorch_matrix = torch::rand({3, 3}, torch::kFloat32);
torch::Tensor libtorch_vector = torch::rand({3}, torch::kFloat32);

std::cout << libtorch_matrix << std::endl;
std::cout << libtorch_vector << std::endl;

std::cout << torch::matmul(libtorch_matrix, libtorch_vector) << std::endl;

 0.5997  0.7039  0.5234
 0.9988  0.1635  0.3138
 0.5716  0.2546  0.6526
[ CPUFloatType{3,3} ]
 0.5511
 0.6072
 0.1811
[ CPUFloatType{3} ]
 0.8527
 0.7066
 0.5878
[ CPUFloatType{3} ]


In [5]:
void tensor_vector_to_std_vector(const torch::Tensor &tensor, std::vector<float> &vector)
{
    TORCH_CHECK(tensor.is_contiguous(), "Tensor must be contiguous");
    int length = tensor.numel();
    vector.resize(length);
    std::memcpy(vector.data(), tensor.data_ptr<float>(), length * sizeof(float));
}

In [6]:
void tensor_2d_matrix_to_std_vector(const torch::Tensor &tensor, std::vector<float> &matrix, int &rows, int &cols)
{
    TORCH_CHECK(tensor.is_contiguous(), "Tensor must be contiguous");
    rows = tensor.size(0);
    cols = tensor.size(1);
    matrix.resize(rows * cols);
    std::memcpy(matrix.data(), tensor.data_ptr<float>(), rows * cols * sizeof(float));
}

In [7]:
void debug_flatten_matrix(const std::vector<float> &flattened_matrix, int num_cols, const std::string &name = "")
{
    if (!name.empty())
    {
        std::cout << name << ":\n";
    }

    int num_rows = flattened_matrix.size() / num_cols;

    std::cout << "[\n";
    for (int i = 0; i < num_rows; ++i)
    {
        std::cout << "  [";
        for (int j = 0; j < num_cols; ++j)
        {
            // std::cout << std::setw(10) << flattened_matrix[i * num_cols + j] << " ";
            std::cout << flattened_matrix[i * num_cols + j];
            if (j < num_cols - 1)
            {
                std::cout << ", ";
            }
        }
        std::cout << "]";
        if (i < num_rows - 1)
        {
            std::cout << ",\n";
        }
    }
    std::cout << "\n]" << std::endl;
    std::cout << "(size: " << flattened_matrix.size() << ")" << std::endl;
    std::cout << "(convert size: " << num_rows << ", " << num_cols << ")" << std::endl;
}


In [8]:
void debug_vector(const std::vector<float> &vec, const std::string &name = "")
{
    if (!name.empty())
    {
        std::cout << name << ": ";
    }
    std::cout << "[";
    for (size_t i = 0; i < vec.size(); ++i)
    {
        std::cout << vec[i];
        if (i < vec.size() - 1)
        {
            std::cout << ", ";
        }
    }
    std::cout << "]" << std::endl;
    std::cout << "(size: " << vec.size() << ")" << std::endl;
}

In [9]:
std::vector<float> std_vector_matrix;
int rows, cols;
tensor_2d_matrix_to_std_vector(libtorch_matrix, std_vector_matrix, rows, cols);
debug_flatten_matrix(std_vector_matrix, cols);

std::vector<float> std_vector_vector;
tensor_vector_to_std_vector(libtorch_vector, std_vector_vector);
debug_vector(std_vector_vector);

[
  [0.59971, 0.703899, 0.523357],
  [0.998771, 0.163468, 0.31378],
  [0.57161, 0.25458, 0.652616]
]
(size: 9)
(convert size: 3, 3)
[0.551134, 0.607241, 0.181109]
(size: 3)


In [10]:
#pragma cling add_include_path("/opt/conda/pkgs/mkl-include-2023.2.0-h84fe81f_50496/include")
#pragma cling add_library_path("/opt/conda/pkgs/mkl-2023.2.0-h84fe81f_50496/lib")

In [11]:
#include <mkl.h>
std::vector<float> std_vector_result(3); // Need to specify size otherwise will crash
cblas_sgemv(CblasRowMajor, CblasNoTrans, rows, cols, 1.0, std_vector_matrix.data(), cols, std_vector_vector.data(), 1, 0.0, std_vector_result.data(), 1);
debug_vector(std_vector_result);

[0.852742, 0.70655, 0.58782]
(size: 3)


In [1]:
#include <xtensor/xarray.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xrandom.hpp>
// Create a 3x3 matrix with random float values
xt::xarray<float> xtensor_matrix = xt::random::rand<float>({3, 3});

// Create a vector of length 3 with random float values
xt::xarray<float> xtensor_vector = xt::random::rand<float>({3});

std::cout << xtensor_matrix << std::endl;
std::cout << xtensor_vector << std::endl;

{{ 0.814724,  0.135477,  0.905792},
 { 0.835009,  0.126987,  0.968868},
 { 0.913376,  0.221034,  0.632359}}
{ 0.308167,  0.09754 ,  0.547221}


In [2]:
// BUG: xlinalg.hpp seem conflict with mkl (this was running after reset kernel)
#include <cstddef>
#include <xtensor-blas/xlinalg.hpp>
std::cout << xt::linalg::dot(xtensor_matrix, xtensor_vector) << std::endl;

{ 0.759953,  0.799893,  0.649072}


In [3]:
// https://xtensor.readthedocs.io/en/latest/adaptor.html
#include <cstddef>
#include <xtensor/xadapt.hpp>

std::vector<double> v = {1., 2., 3., 4., 5., 6. };
std::vector<std::size_t> shape = { 2, 3 };
auto a1 = xt::adapt(v, shape);

xt::xarray<double> a2 = {{ 1., 2., 3.},
                         { 4., 5., 6.}};

xt::xarray<double> res = a1 + a2;
std::cout << res << std::endl;

{{  2.,   4.,   6.},
 {  8.,  10.,  12.}}
